# LLM Iteration 20

**Aim**

To change the random seeds being used in the multiple replications function to ensure different outputs each time.

**Prompt**

Consider the previously output Scenario class:
```python
class Scenario:
    def __init__(self, num_beds):
        self.num_beds = num_beds
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self):
        interarrival_distributions = {}
        for patient_type, seed in self.random_seeds.items():
            interarrival_distributions[patient_type] = \
                np.random.default_rng(seed)
        return interarrival_distributions
```

This means that each time the multiple_replications function is run, the same output appears due to the random seeds being used. Create a new function that generates random number generator set depending on the number of runs.

**Note**

Modifications have been made to the output from the LLM, indicated by:
```python
########## MODIFICATION ##########
change
##################################
```
This is either used to add in code from previous iterations if only certain sections were updated, used to correct the LLM's code if required, or used to check the model is working as expected.

**Additional code to control output**

In [1]:
def trace(msg):
    '''
    Utility function for printing simulation.
    Set the TRACE constant to FALSE to turn tracing off.
    
    Params:
    -------
    msg: str
        string to print to screen.
    '''
    if TRACE:
        print(msg)

**LLM Output**

To address the issue of repetitive output in the multiple_replications function due to the fixed random seeds, you can create a new function that generates a unique set of random number generators based on the number of runs. This will ensure variability in the simulation outcomes for each replication. Here's how you can modify the code:

In [2]:
import simpy
import numpy as np
import math
import pandas as pd

In [3]:
class Scenario:
    def __init__(self, num_beds):
        self.num_beds = num_beds
        self.default_interarrival_times = {
            "Stroke": 1.2,
            "TIA": 9.3,
            "Complex Neurological": 3.6,
            "Other": 3.2
        }
        self.random_seeds = {
            "Stroke": 12345,
            "TIA": 54321,
            "Complex Neurological": 98765,
            "Other": 67890
        }

    def create_distributions(self, num_runs):
        interarrival_distributions = {}
        seeds = self.generate_seeds(num_runs)
        for patient_type, seed in seeds.items():
            interarrival_distributions[patient_type] = np.random.\
                default_rng(seed)
        return interarrival_distributions

    def generate_seeds(self, num_runs):
        seeds = {}
        for patient_type, seed in self.random_seeds.items():
            seeds[patient_type] = [seed + i for i in range(num_runs)]
        return seeds

In [4]:
class StrokeWard:
    def __init__(self, scenario):
        self.env = simpy.Environment()
        self.beds = simpy.Resource(self.env, capacity=scenario.num_beds)
        self.patient_count = 0
        self.wait_times = []
        self.waiting_patients = []
        self.length_of_stay = {
            "Stroke": {"mean": 7.4, "std_dev": 8.6},
            "TIA": {"mean": 1.8, "std_dev": 2.3},
            "Complex Neurological": {"mean": 4.0, "std_dev": 5.0},
            "Other": {"mean": 3.8, "std_dev": 5.2}
        }
        self.interarrival_distributions = scenario.\
            create_distributions(num_runs=1)
        
        self.route_probabilities = {
            "Stroke": {"ESD": 0.13, "Rehab Unit": 0.24, "Other": 0.63},
            "TIA": {"ESD": 0.01, "Rehab Unit": 0.01, "Other": 0.98},
            "Complex Neurological": {"ESD": 0.05, "Rehab Unit": 0.11,
                                     "Other": 0.84},
            "Other": {"ESD": 0.10, "Rehab Unit": 0.05, "Other": 0.85}
        }

    # Remaining methods remain unchanged

        ########## MODIFICATION ##########
        self.patients_in_beds = 0
        self.bed_in_use_store = []
        self.los_store = []
        self.patient_type_store = []
        ##################################

    #### MODIFICATION - code from previous iter ####
    def admit_patient(self, patient):
        arrival_time = round(self.env.now, 2)
        with self.beds.request() as bed:
            yield bed
            ########## MODIFICATION ##########
            bed_arrival_time = self.env.now
            self.patients_in_beds += 1
            self.bed_in_use_store.append(self.patients_in_beds)
            self.patient_type_store.append(patient.patient_type)
            ##################################

            wait_time = max(0, round(self.env.now - arrival_time, 2))
            self.wait_times.append(wait_time)
            self.waiting_patients.append(len(self.beds.queue))
            ########## MODIFICATION - added trace function to print #######
            ########## to save space ######################################
            trace(f'Patient {patient.id} ({patient.patient_type}) admitted '
                  f'to the stroke ward at day {round(self.env.now, 2)}. '
                  f'Wait time: {wait_time} days')
            mu, sigma = calculate_mu_sigma(self.length_of_stay[patient.\
                                           patient_type]["mean"],
                                           self.length_of_stay[patient.\
                                           patient_type]["std_dev"])
            los = np.random.default_rng(patient.seed).lognormal(mu, sigma)
            yield self.env.timeout(round(los, 2))
            ########## MODIFICATION ##########
            self.patients_in_beds -= 1
            bed_leave_time = self.env.now
            self.los_store.append(round(bed_leave_time - bed_arrival_time, 2))
            ##################################

            trace(f'Patient {patient.id} ({patient.patient_type}) discharged '
                  f'from the stroke ward at day {round(self.env.now, 2)}')

            route_probabilities = list(self.route_probabilities[patient.\
                                       patient_type].values())
            route_names = list(self.route_probabilities[patient.\
                               patient_type].keys())
            
            rng = np.random.default_rng(patient.seed)
            route_choice = rng.choice(route_names, p=route_probabilities)
            
            trace(f'Patient {patient.id} ({patient.patient_type}) routed to: '
                  f'{route_choice}')

    def patient_arrival(self, patient_type):
        while True:
            if patient_type in self.interarrival_distributions:
                interarrival_time = self.\
                    interarrival_distributions[patient_type].\
                    exponential(scenario.\
                                default_interarrival_times[patient_type])
                yield self.env.timeout(interarrival_time)
                patient = Patient(self.patient_count, patient_type,
                                  self.\
                                  interarrival_distributions[patient_type])
                self.patient_count += 1
                self.env.process(self.admit_patient(patient))

    def summary(self):
        mean_wait_time = round(np.mean(self.wait_times), 2)
        mean_waiting_patients = round(np.mean(self.waiting_patients), 2)
        
        summary_data = {
            'Mean Wait Time': [mean_wait_time],
            'Mean Waiting Patients': [mean_waiting_patients]
        }
        
        summary_df = pd.DataFrame(summary_data)
        
        print("Summary:")
        print(summary_df)
    ################################################

In [5]:
def single_run(scenario):
    stroke_ward = StrokeWard(scenario)

    PATIENTS = ["Stroke", "TIA", "Complex Neurological", "Other"]
    for patient_type in PATIENTS:
        stroke_ward.env.process(stroke_ward.patient_arrival(patient_type))

    run_time = round(60 * 2) # Set the run length to 2 months (default)

    trace("Simulation started...")
    stroke_ward.env.run(until=run_time)
    trace("Simulation ended.")

    stroke_ward.summary()

    ########## MODIFICATION ##########
    return stroke_ward
    ##################################

def multiple_replications(scenario, num_runs):
    ########## MODIFICATION ##########
    models = []
    ##################################
    for i in range(num_runs):
        scenario_with_seeds = Scenario(scenario.num_beds)
        scenario_with_seeds.random_seeds = {k: v[i] for k, v in scenario.\
                                            generate_seeds(num_runs).items()}
        
        stroke_ward = single_run(scenario_with_seeds)
    ########## MODIFICATION ##########
        models.append(stroke_ward)

    return models
    ##################################

In [6]:
#### MODIFICATION - code from previous iter ####
class Patient:
    def __init__(self, id, patient_type, seed):
        self.id = id
        self.patient_type = patient_type
        self.seed = seed

def calculate_mu_sigma(mean, std_dev):
    v = std_dev ** 2
    m = mean
    phi = math.sqrt(v + m ** 2)
    mu = math.log((m ** 2) / phi)
    sigma = math.sqrt(math.log((phi ** 2) / (m ** 2)))
    return mu, sigma
################################################

In [7]:
########## MODIFICATION ##########
TRACE = False
##################################

# Example usage:
scenario = Scenario(num_beds=10)

# Run a single simulation
stroke_ward = single_run(scenario)

# Run multiple replications with varying random seeds
num_replications = 5
models = multiple_replications(scenario, num_replications)

Summary:
   Mean Wait Time  Mean Waiting Patients
0            1.44                    2.1
Summary:
   Mean Wait Time  Mean Waiting Patients
0            1.44                    2.1
Summary:
   Mean Wait Time  Mean Waiting Patients
0            1.74                   2.92
Summary:
   Mean Wait Time  Mean Waiting Patients
0            2.04                   2.57
Summary:
   Mean Wait Time  Mean Waiting Patients
0            1.71                    2.5
Summary:
   Mean Wait Time  Mean Waiting Patients
0            0.81                   1.01


In this updated code snippet:
- The Scenario class now includes a method generate_seeds that generates unique sets of random seeds based on the number of runs.
- The create_distributions method now uses these generated seeds to create different sets of random number generators for each replication.
- In the multiple_replications function, a new instance of Scenario is created for each run with updated random seeds from the generated sets.
- This approach ensures that each replication uses a distinct set of random seeds, introducing variability in the simulation outcomes across multiple runs.

**Checks**

In [8]:
class TestCase:
    def __init__(self, scenario):
        self.scenario = scenario

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        ward = single_run(self.scenario)

        if ward.env.now > round(60 * 2):
            print('The simulation has continued for longer than input.')

    def test_bed_hold(self):
        '''
        Checks that patients are holding beds until they leave the model.
        '''
        ward = single_run(self.scenario)

        for bed_use in ward.bed_in_use_store:
            if bed_use > self.scenario.num_beds:
                print('The number of beds in use exceeded limit.')

    def test_length_of_stay(self):
        '''
        Checks that specified length of stay correctly implemented.
        '''
        ward = single_run(self.scenario)

        print('\nLength of stay:')
        for pat_type, los in zip(ward.patient_type_store, ward.los_store):
            print(f'({pat_type}) {los}')

In [9]:
TRACE = False
scenario = Scenario(num_beds=10)

test = TestCase(scenario)
test.test_run_length()

Summary:
   Mean Wait Time  Mean Waiting Patients
0            1.44                    2.1


In [10]:
TRACE = True
scenario = Scenario(num_beds=1)

test = TestCase(scenario)
test.test_bed_hold()

Simulation started...
Patient 0 (Complex Neurological) admitted to the stroke ward at day 0.09. Wait time: 0 days
Patient 0 (Complex Neurological) discharged from the stroke ward at day 0.49
Patient 0 (Complex Neurological) routed to: Other
Patient 1 (Stroke) admitted to the stroke ward at day 0.49. Wait time: 0.27 days
Patient 1 (Stroke) discharged from the stroke ward at day 2.65
Patient 1 (Stroke) routed to: Other
Patient 2 (Stroke) admitted to the stroke ward at day 2.65. Wait time: 1.66 days
Patient 2 (Stroke) discharged from the stroke ward at day 11.44
Patient 2 (Stroke) routed to: Other
Patient 3 (Stroke) admitted to the stroke ward at day 11.44. Wait time: 9.94 days
Patient 3 (Stroke) discharged from the stroke ward at day 22.55
Patient 3 (Stroke) routed to: ESD
Patient 4 (TIA) admitted to the stroke ward at day 22.55. Wait time: 20.53 days
Patient 4 (TIA) discharged from the stroke ward at day 24.53
Patient 4 (TIA) routed to: Other
Patient 5 (Stroke) admitted to the stroke wa

In [11]:
TRACE = False
scenario = Scenario(num_beds=10)

test = TestCase(scenario)
test.test_length_of_stay()

Summary:
   Mean Wait Time  Mean Waiting Patients
0            1.44                    2.1

Length of stay:
(Complex Neurological) 0.4
(Stroke) 2.16
(Stroke) 1.36
(Stroke) 4.5
(TIA) 0.73
(Stroke) 4.25
(Stroke) 1.18
(Other) 1.23
(Complex Neurological) 2.39
(Other) 1.55
(Complex Neurological) 3.46
(Complex Neurological) 1.18
(Other) 1.54
(Stroke) 2.68
(Other) 4.17
(Stroke) 3.78
(Stroke) 9.53
(Complex Neurological) 1.55
(Stroke) 8.22
(Other) 1.58
(Stroke) 10.01
(Complex Neurological) 2.65
(Complex Neurological) 2.63
(Stroke) 1.73
(Other) 11.12
(Stroke) 2.81
(Other) 1.85
(Stroke) 6.12
(Stroke) 7.96
(Stroke) 16.39
(Stroke) 5.15
(TIA) 5.94
(Stroke) 6.37
(Stroke) 6.18
(Stroke) 2.18
(Complex Neurological) 5.04
(Other) 1.64
(Stroke) 7.48
(Stroke) 15.61
(Stroke) 23.79
(Complex Neurological) 1.33
(Stroke) 4.1
(Stroke) 0.39
(Complex Neurological) 2.62
(Other) 4.86
(Stroke) 23.78
(Other) 2.52
(Stroke) 4.08
(Complex Neurological) 0.94
(Stroke) 16.77
(Complex Neurological) 0.81
(Stroke) 3.7
(Complex 

In [13]:
# Checking multiple replication results if number of beds are changed
TRACE = False

scenario = Scenario(num_beds=1)
models = multiple_replications(scenario, num_runs=5)

Summary:
   Mean Wait Time  Mean Waiting Patients
0           50.45                   81.0
Summary:
   Mean Wait Time  Mean Waiting Patients
0           57.84                  91.23
Summary:
   Mean Wait Time  Mean Waiting Patients
0            41.6                  66.87
Summary:
   Mean Wait Time  Mean Waiting Patients
0           39.28                  59.74
Summary:
   Mean Wait Time  Mean Waiting Patients
0           63.48                  98.26


In [16]:
# Checking the output of the generate seeds method
scenario = Scenario(num_beds=10)
test_seeds = scenario.generate_seeds(num_runs=5)
for name, seeds in test_seeds.items():
    print(f'{name}: seeds => {seeds}')

Stroke: seeds => [12345, 12346, 12347, 12348, 12349]
TIA: seeds => [54321, 54322, 54323, 54324, 54325]
Complex Neurological: seeds => [98765, 98766, 98767, 98768, 98769]
Other: seeds => [67890, 67891, 67892, 67893, 67894]


**Reflection**

The LLM has created a function inside the Scenario class that changes the random seeds based on the number of iterations as instructed, and now the output is different for each multiple replication. A Scenario class object is being passed to the multiple replications function, but is only being used to sample the number of beds used, when this could be passed to the function itself. The new method inside the Scenario class enables different random seeds to be generated that add 1 to the seed each time, as shown in the test above. Patients are still holding their bed until they leave and their length of stay is being sampled from distributions. As the run length is being set up inside the function, it cannot be changed unless the function itself is altered.